Jorge Arboleya Carrio, David Esparza Sainz, Cristina Gómez Calvo

### Ejemplo 3. CliffWalking  (para entregar) 

Para esta práctica se pide entrenar y analizar en detalle el comportamiento de un agente para el problema CliffWalking (y/o BlackJack).
Puedes encontrar más información en la documentación de Gymnasium:
[https://gymnasium.farama.org/environments/toy_text/cliff_walking/]

El juego comienza con el jugador en la ubicación (3, 0) de una cuadrícula de 4x12, con el objetivo situado en (3, 11). Si el jugador llega al objetivo, el episodio termina.
Un precipicio se extiende a lo largo de (3, 1..10). Si el jugador se mueve a una ubicación del precipicio, vuelve al punto de inicio.

El jugador realiza movimientos hasta llegar al objetivo.

Adaptado del Ejemplo 6.6 (página 132) del libro Reinforcement Learning: An Introduction de Sutton y Barto.
Según el problema original (y la documentación) el precipicio puede configurarse como resbaladizo (desactivado por defecto), por lo que el jugador puede moverse perpendicularmente a la dirección deseada en ocasiones (ver is_slippery). Sin embargo, no está disponible la opción en la versión v0 proporcionada por gymnasium, por lo que no usaremos la opción.

In [1]:
import gymnasium as gym
problema = gym.make("CliffWalking-v0", render_mode="human")

In [2]:
print("Tamaño de espacio de estados", problema.observation_space)
print("Estado aleatorio", problema.observation_space.sample())
size_estados = problema.observation_space.n
print("Hay", size_estados, " estados posibles.")
problema.reset()
problema.render()
env=problema

Tamaño de espacio de estados Discrete(48)
Estado aleatorio 45
Hay 48  estados posibles.


In [ ]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info,_  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

¿Qué tal funciona la resolución con acciones aleatorias? 

Entrena un agente que resuelva el problema y comenta en detalle los resultados modificando los valores de los párametros, el número de episodios de aprendizaje, la recompensa por defecto. Define una métrica adecuada para el problema y evalua el aprendizaje para cada caso. 
Indica al final cual es la configuración de parámetros elegida. Al final del notebook se dan más detalles en **Actividad para Entregar**.

## Actividad para entregar

Configura el aprendizaje por refuerzo para resolver alguno/s de los problemas Toy Text de Gymnasium utilizando distintos valores de configuración de los parámetros y observa qué configuración se comporta mejor.
     
    1. Define las métricas adecuadas para poder evaluar qué tal se comporta el agente entrenado con distintas configuraciones de parámetros.
    2. Realiza una variación dinámica de los valores de los parámetros durante el entrenamiento para que los valores no sean fijos 
    2. Modifica la recompensa por defecto para ver cómo afecta al entrenamiento. Observa cuál es la función de recompensa que se define por defecto (consulta en la documentación de gym). Mejórala reescribiendo el valor de reward y observa cómo afecta la mejora de la función de recompensa en el proceso de aprendizaje. Explica cómo has medido esta mejora.
    4. Realiza pruebas y saca conclusiones claras justificadas con el resumen de resultados. Puedes usar gráficas.  


### Apartado 1

Vamos a evaluar el funcionamiento de nuestro programa en base a las siguientes métricas:
1. Número de pasos promedio.
2. Número de veces que se tira por el precipio en promedio.

### Apartado 2

Para modificar dinámicamente los parámetros, aplicaremos la siguiente fórmula: sea x_0 cualquiera de nuestros parámetros iniciales (α, γ ó ε):
x_n = x_{n-1} - val, donde val= x_0*1000/numEpisodios. De esta forma, iremos restando cada 1000 pasos una cantidad proporcional fija, de tal forma que si el número de episodios es mayor que 1000, acaban siendo 0 (en caso contrario, consideramos que no hay un número suficiente de episodios como para disminuir alguno de los parámetros).

Entrenamiento del agente

In [1]:
# Primero se inicializa la Q-table a matrix of zeros:
import gymnasium as gym
env = gym.make("CliffWalking-v0")
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [2]:
state, _ = env.reset()
print(state)

36


In [3]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
episodes = 270
valAlpha = alpha*1000/episodes
valGamma = gamma*1000/episodes
valEpsilon = epsilon*1000/episodes
# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, episodes+1):
    state,_ = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, _ = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -100:
            penalties += 1

        state = next_state
        epochs += 1
    #Modificamos el valor de los parámetros cada 1000 pasos.
    if i%1000 == 0 and i < episodes:
        alpha = alpha - valAlpha
        gamma = gamma - valGamma
        epsilon = epsilon - valEpsilon
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episodio: {i}")
        print(f"Valor de alpha:{alpha}")
        print(f"Valor de gamma:{gamma}")
        print(f"Valor de epsilon:{epsilon}")
        
print("Training finished.\n")


Episodio: 200
Valor de alpha:0.1
Valor de gamma:0.6
Valor de epsilon:0.1
Training finished.

CPU times: total: 109 ms
Wall time: 117 ms


In [4]:
total_epochs, total_penalties = 0, 0
episodes = 3

for i in range(episodes):
    print(f"Episodio: {i}")
    state,_ = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[int(state)])
        state, reward, done, info, _ = env.step(action)

        if reward == -100:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    
print(f"Resultados después de {episodes} episodios:")
print(f"Pasos promedio por episodio: {total_epochs / episodes}")
print(f"Número de caídas al precipicio promedio por episodio: {total_penalties / episodes}")

Episodio: 0
Episodio: 1
Episodio: 2
Resultados después de 3 episodios:
Pasos promedio por episodio: 13.0
Número de caídas al precipicio promedio por episodio: 0.0


Como podemos observar, después de haberlo entrenado el muñeco llega a la posición objetivo en un promedio de 13 pasos, lo cual es óptimo. Esto se debe a que tiene que ir una casilla hacia arriba, 11 a la derecha y una hacia abajo. Además, de acuerdo con nuestra segunda métrica, observamos que en promedio no se cae ninguna vez por el precipicio.

### Apartado 3

In [86]:
# Primero se inicializa la Q-table a matrix of zeros:
import gymnasium as gym
env = gym.make("CliffWalking-v0")
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

state, _ = env.reset()
print(state)


36


In [87]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
episodes = 270
valAlpha = alpha*1000/episodes
valGamma = gamma*1000/episodes
valEpsilon = epsilon*1000/episodes
# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, episodes+1):
    state,_ = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, _ = env.step(action) 

        if reward == -100: #Si cae por el precipicio, modificamos la recompensa a -1
            reward=-1
            penalties += 1
        if reward==0: #Si está en una celda normal, modificamos la recompensa a 11 para que se note el cambio de recompensa en el caso de caer por el precipicio.
            reward=100
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = next_state
        epochs += 1
    #Modificamos el valor de los parámetros cada 1000 pasos.
    if i%1000 == 0 and i < episodes:
        alpha = alpha - valAlpha
        gamma = gamma - valGamma
        epsilon = epsilon - valEpsilon
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episodio: {i}")
        print(f"Valor de alpha:{alpha}")
        print(f"Valor de gamma:{gamma}")
        print(f"Valor de epsilon:{epsilon}")
        
print("Training finished.\n")


Episodio: 200
Valor de alpha:0.1
Valor de gamma:0.6
Valor de epsilon:0.1
Training finished.

CPU times: total: 156 ms
Wall time: 140 ms


In [88]:
total_epochs, total_penalties = 0, 0
episodes = 3

for i in range(episodes):
    state,_ = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[int(state)])
        state, reward, done, info, _ = env.step(action)

        if reward == -100:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    
print(f"Resultados después de {episodes} episodios:")
print(f"Pasos promedio por episodio: {total_epochs / episodes}")
print(f"Número de caídas al precipicio promedio por episodio: {total_penalties / episodes}")

Resultados después de 3 episodios:
Pasos promedio por episodio: 13.0
Número de caídas al precipicio promedio por episodio: 0.0


Observamos que, cambiando los episodios de entrenamiento a 270 y los de prueba a 3 nos da un promedio siempre de 13 pasos sin cambiar la recompensa. Sin embargo, en la versión en la que hemos modificado la recompensa, hemos probado a ejecutarlo varias veces, y a veces sale igual que en la versión sin modificar las recompensas, pero en otras se queda pillado al no haber aprendido lo suficiente y en otras nos sale que el número promedio de pasos es mayor.


### Apartado 4

Hemos realizado diferentes pruebas y hemos observado que, una vez aprendido el camino óptimo (13 pasos), no vuelve a elegir otro camino ni se cae por el precipicio, debido a que siempre son el mismo origen y destino. Hemos reducido también el número de episodios de entrenamiento y hemos observado que en relativamente pocos episodios encuentra el camino adecuado y a partir de ese momento lo reproduce siempre. También hemos probado a modificar las recompensas y hemos visto que en la mayoría de los casos sigue haciendo el recorrido óptimo, pero alguna vez necesita más episodios de entrenamiento (en comparación con las recompensas originales). Por lo tanto, consideramos que en este problema no hay una diferencia significativa al modificar las recompensas porque no hay muchos parámetros a tener en cuenta, sólo evitar el precipicio y llegar al destino lo antes posibles. Con relación a la modificación dinámica de parámetros, tampoco notamos un impacto significativo debido a que, como hemos comentado anteriormente, se entrena relativamente rápido incluso sin modificación dinámica.
En conclusión, al ser un problema en el que no se tienen en cuenta muchos parámetros, ninguna de las modificaciones realizadas es muy notoria.